In [ ]:
!pip install datasets

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('glue', 'sst2')

# Use a small subset of the dataset (e.g., first 100 samples)
small_train_dataset = dataset['train'].select(range(1000))
small_validation_dataset = dataset['validation'].select(range(100))

# Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Define the training function
def train(model, tokenizer, dataset, epochs=1):  # Set epochs to 1
    # Define the optimizer
    optimizer = AdamW(model.parameters(), lr=2e-5)

    # Train the model
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        for batch in dataset:
            # Tokenize the input
            inputs = tokenizer(batch['sentence'], return_tensors='pt', padding=True, truncation=True)

            # Get the labels
            labels = torch.tensor(batch['label'], dtype=torch.long).unsqueeze(0)  # Add batch dimension

            # Perform a forward pass
            outputs = model(**inputs, labels=labels)

            # Calculate the loss
            loss = outputs.loss

            # Perform a backward pass
            loss.backward()

            # Update the weights
            optimizer.step()

            # Zero the gradients
            optimizer.zero_grad()

        # Evaluate the model after each epoch
        evaluate(model, tokenizer, small_validation_dataset)

# Define the evaluation function
def evaluate(model, tokenizer, dataset):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for batch in dataset:
            # Tokenize the input
            inputs = tokenizer(batch['sentence'], return_tensors='pt', padding=True, truncation=True)

            # Get the labels
            labels = torch.tensor(batch['label'], dtype=torch.long).unsqueeze(0)  # Add batch dimension

            # Perform a forward pass
            outputs = model(**inputs, labels=labels)

            # Calculate the loss
            loss = outputs.loss
            total_loss += loss.item()

            # Calculate the accuracy
            accuracy = (outputs.logits.argmax(-1) == labels).float().mean()
            total_accuracy += accuracy.item()

    # Print the average loss and accuracy
    avg_loss = total_loss / len(dataset)
    avg_accuracy = total_accuracy / len(dataset)
    print(f'Average Loss: {avg_loss:.4f}, Average Accuracy: {avg_accuracy:.4f}')

# Train the model
train(model, tokenizer, small_train_dataset)

# Save the trained model
model.save_pretrained('trained_model')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Validation Accuracy: 0.7700


['logistic_regression_sentiment.pkl']

In [ ]:
# Save the trained model
model.save_pretrained('trained_model')
tokenizer.save_pretrained('trained_model')

('trained_model/tokenizer_config.json',
 'trained_model/special_tokens_map.json',
 'trained_model/vocab.txt',
 'trained_model/added_tokens.json')